# **UNIVERSIDAD TORCUATO DI TELLA**
## **MAESTRÍA EN ECONOMETRÍA**

---

### **TRABAJO PRÁCTICO DE ECONOMETRÍA**

- **Profesor:** González-Rozada, Martín  
- **Ayudante:** Lening, Iara  
- **Alumno:** Guzzi, David Alexander  (Legajo n°: 24H1970, DNI: 37.703.649)  

**Ciclo Lectivo:** Tercer Trimestre, 2024  

---

##### **1. PROPIEDADES DE MUESTRA FINITA DE FGLS (MCGE).**

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

np.random.seed(3649)

def fgls_estimation(x: np.ndarray, y: np.ndarray) -> tuple[float, float, float]:

    """
    Realiza una estimación de Feasible Generalized Least Squares (FGLS para un modelo lineal 
    con errores heterocedásticos.

    Parámetros:
    x : array-like
        Variable regresora.
    y : array-like
        Variable de respuesta.

    Retorna:
    beta0_fgls : float
        Intercepto estimado.
    beta1_fgls : float
        Coeficiente estimado para el regresor.
    se_beta1_fgls : float
        Error estándar del coeficiente estimado para el regresor.
    """

    # 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo.
    X = np.column_stack((np.ones_like(x), x)) 
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y 

    # 2. Calcular los residuos del modelo, elevarlos al cuadrado y obtener el logaritmo de los mismos.
    u_hat = y - X @ beta_ols
    u_hat2 = u_hat ** 2  
    log_u_hat2 = np.log(u_hat2)

    # 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x + x2, se estima por OLS usando u_hat2 como proxy de sigma2.
    X_aux = np.column_stack((x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ log_u_hat2
    
    # 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes).
    log_sigma2_hat = X_aux @ gamma_hat
    sigma2_hat = np.exp(log_sigma2_hat)

    # 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por sigma2_hat y estimar por OLS.
    X_aux2 = np.column_stack((x / sigma2_hat,  x**2 / sigma2_hat))
    u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat
    gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

    # 6. Usar las estimaciones de la segunda regresión auxiliar y obtener las varianzas ajustadas (consistentes).
    sigma2_tilde = np.maximum(X_aux @ gamma_hat2, 1e-6) #Se asegura positividad de las varianzas. 
    omega_tilde_inv = np.linalg.inv(np.diag(sigma2_tilde)) #Se construye matriz omega inversa.  
    
    # 7. Aplicar GLS en la regresión de y ~ x, utilizando como ponderador la matriz omega estimada.
    # Obs.: También se podría usar uno sobre la raíz cuadrada de sigma2 tilde como ponderador en la regresión de y ~ x.
    
    # Se obtienen las estimaciones de los parámetros.
    beta_fgls = np.linalg.inv(X.T @ omega_tilde_inv @ X) @ X.T @ omega_tilde_inv @ y
    
    # Se obtienen las estimaciones de los errores estándar de los parámetros.
    residuals_fgls = y - X @ beta_fgls
    s2_fgls = (residuals_fgls.T @ omega_tilde_inv @ residuals_fgls) / ( X.shape[0] - X.shape[1])
    var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_tilde_inv @ X)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]   
    
    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [3]:
def simulate_fgls(n_samples: int, n_observations_list: list[int], beta_0_true: float, beta_1_true: float, beta1_H0: float) -> pd.DataFrame:
    
    """
    Simula la estimación de FGLS para distintos tamaños de muestra y evalúa la validez de un test de hipótesis.

    Parameters:
    n_samples : int
        Número de simulaciones a realizar para cada tamaño de muestra.
    n_observations_list : list of int
        Lista con los distintos tamaños de muestra a evaluar.
    beta_0_true : float
        Valor verdadero del intercepto en la simulación.
    beta_1_true : float
        Valor verdadero del coeficiente del regresor en la simulación.
    beta1_H0 : float
        Valor hipotético de beta_1 bajo H0 para el test de hipótesis.

    Returns:
    results_df : pandas.DataFrame
        DataFrame con estadísticas de los coeficientes estimados y tasas de rechazo del test de hipótesis.
        Columnas:
        - 'n_observations': Tamaño de la muestra.
        - 'mean_beta0': Media de las estimaciones de beta_0.
        - 'median_beta0': Mediana de las estimaciones de beta_0.
        - 'std_beta0': Desviación estándar de las estimaciones de beta_0.
        - 'mean_beta1': Media de las estimaciones de beta_1.
        - 'median_beta1': Mediana de las estimaciones de beta_1.
        - 'std_beta1': Desviación estándar de las estimaciones de beta_1.
        - 'test_size_1pct': Proporción de rechazos de H0 al 1% de significancia.
        - 'test_size_5pct': Proporción de rechazos de H0 al 5% de significancia.
    """

    
    results = []
    for n_obs in n_observations_list:
        sample_results = []
        
        for i in range(n_samples):
            # Generación de datos.
            x = np.random.uniform(1, 50, n_obs) # x ~ U[1, 50]
            u = np.random.normal(scale=x) # u ~ N(0, omega) con omega = x (errores heterocedásticos)
            y = beta_0_true + beta_1_true * x + u
            
            # Estimación FGLS.
            beta_0_hat, beta_1_hat, se_beta1_hat = fgls_estimation(x, y)
            
            # Test de hipótesis para beta_1 = 0.8.
            t_stat = (beta_1_hat - beta1_H0) / se_beta1_hat
            p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
            reject_1pct = p_value < 0.01 # Rechazar H0 al 1% de significancia.
            reject_5pct = p_value < 0.05 # Rechazar H0 al 5% de significancia.
            
            sample_results.append([beta_0_hat, beta_1_hat, reject_1pct, reject_5pct])
        
        # Convertir a DataFrame.
        df_results = pd.DataFrame(sample_results, columns=['beta_0_hat', 'beta_1_hat', 'reject_1pct', 'reject_5pct'])
        
        # Calcular estadísticas.
        mean_beta0 = df_results['beta_0_hat'].mean()
        mean_beta1 = df_results['beta_1_hat'].mean()
        median_beta0 = df_results['beta_0_hat'].median()
        median_beta1 = df_results['beta_1_hat'].median()
        std_beta0 = df_results['beta_0_hat'].std()
        std_beta1 = df_results['beta_1_hat'].std()
        test_size_1pct = df_results['reject_1pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 1% de significancia.
        test_size_5pct = df_results['reject_5pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 5% de significancia.
        
        results.append([n_obs, mean_beta0, median_beta0, std_beta0, mean_beta1, median_beta1, std_beta1, test_size_1pct, test_size_5pct])
    
    results_df = pd.DataFrame(results, columns=['n_observations', 'mean_beta0', 'median_beta0', 'std_beta0', 'mean_beta1', 'median_beta1', 'std_beta1', 'test_size_1pct', 'test_size_5pct'])
    
    return results_df

In [ ]:
# Parámetros iniciales
test_results = simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.8)
test_results